<a href="https://colab.research.google.com/github/mmillet88/russian_therapy/blob/main/generate_synthetic_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Libraries

In [44]:
!pip install transformers
!pip install sentence-transformers
!pip install transformers sentencepiece datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
import os
import pandas as pd 
import matplotlib.pyplot as plt
import re

from transformers import AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
from transformers import BertForSequenceClassification, AutoTokenizer
from transformers import EarlyStoppingCallback
from transformers import (
    glue_tasks_num_labels,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    GlueDataset,
    GlueDataTrainingArguments,
    TrainingArguments,
)
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import KMeans
import torch
import torch.nn as nn
from torch.nn.functional import cross_entropy
import gc

import datasets
from datasets import load_dataset
from datasets import DatasetDict
from datasets import Dataset

import numpy as np
from numpy import random
from numpy.lib.function_base import average

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

# Load dataset

In [46]:
import os
!mkdir /content/emp_chatbot
!mkdir emp_chatbot/data

mkdir: cannot create directory ‘/content/emp_chatbot’: File exists
mkdir: cannot create directory ‘emp_chatbot/data’: No such file or directory


In [47]:
!ls

all_emotions_responses.json


In [48]:
os.chdir('/content/emp_chatbot/data')

In [49]:
!gdown --id 1uRPD0QPf-I90CopU9eccvzuqxSfsLcPp

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1uRPD0QPf-I90CopU9eccvzuqxSfsLcPp
To: /content/emp_chatbot/data/all_emotions_responses.json
100% 2.38M/2.38M [00:00<00:00, 188MB/s]


In [50]:
data_files = {"train": "all_emotions_responses.json"}
ds = load_dataset('json', data_files=data_files)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-91cd8b96f53e93d8/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [51]:
ds.reset_format()

In [52]:
import pandas as pd
ds.set_format(type="pandas")

In [53]:
ds

DatasetDict({
    train: Dataset({
        features: ['gender', 'age', 'emotion', 'How are you feeling?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Would you like to attempt protocols for joy?'],
        num_rows: 492
    })
})

# Generate utterances


## Define split utterance dictionary with keys as initial questions and additional questions.

In [81]:
split_utterances = dict()
columns = ds['train'].column_names
columns_indices = list(range(9,len(columns)-1))
additional_questions_columns = [columns[index] for index in columns_indices]
joy_question_column = [columns[16]]
initial_questions_indices = [4,5,6,7,8]
initial_questions_columns = [columns[index] for index in initial_questions_indices]

In [82]:
joy_question_column

['Would you like to attempt protocols for joy?']

In [83]:
# check columns identified correctly
print(initial_questions_columns)
print(joy_question_column)
print(additional_questions_columns)

['Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?']
['Would you like to attempt protocols for joy?']
['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?']


In [84]:
# define base questions in russian to be used in a splitting operation
initial_base_questions = ["Вы полагаете, что ваше состояние связано с каким то конкретным событием?", "Эта проблема возникла у вас недавно?", "Методика 6 вызвала у Вас какие-то плохие чувства?", "Методика 11 вызвала у Вас какие-то плохие чувства?", "я задам вам несколько вопросов?"]
joy_question = ['Я бы хотел предложить Вам одно упражнение']
base_questions = ["вы испытываете к кому-либо злость, зависть или агрессию?", "Вы чуствуете ответственность за кого-либо?", "Вам кажется, что вы хотите кого то контролировать?", "Вы испытываете ощущение жертвы и вините кого то в своём состоянии?", "Считаете ли вы себя причиной проблемы?", "Как часто Вы настаиваете на своей позиции не принимая во внимание позицию собеседников?", "Есть ли у вас сложности с друзьями, возможно в личной жизни?"]

In [85]:
split_utterances["base_questions"] = base_questions
split_utterances ['initial_base_questions'] = initial_base_questions
split_utterances['joy_question'] = joy_question

## Split original utterances into empathy and base utterance


In [91]:
# extract emotion names
ds.set_format(type="pandas")
emotions = ds['train'].unique('emotion')
ds.reset_format()

In [87]:
# define a tokenizer for feature extraction
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
pipe = pipeline("feature-extraction", model=model, tokenizer=tokenizer)

In [88]:
# a function to split an utterance into a base and an empathetic part
def split_emp_base(corpus, base_utterance):
  base_embedding = pipe(base_utterance)
  base_utterances = []
  empathy_utterances = []

  for sentence in corpus:
    my_list = re.split('[.!?]', sentence)
    if '' in my_list:
      my_list.remove('')
    simil_list = []
    for sent in my_list:
      simil = util.cos_sim(base_embedding[0][0], pipe(sent)[0][0])
      simil_list.append(simil)
    if len(my_list) >=2:
      if simil_list[0] < simil_list[1]:
        empathy_utterances.append(my_list[0])
        base_utterances.append(my_list[1])
    elif len(my_list)==1:
      base_utterances.append(my_list[0])
  return base_utterances, empathy_utterances

In [97]:
# complete "split_utterances" dictionary with initial/additional base and empathy sentences
for emotion in emotions:
  emotion_filtered = ds.filter(lambda x: x['emotion'] == emotion)
  internal_dict = dict()
  if emotion == "joy":
    additional_questions = 'joy_base_sentences'
    base_question = "joy_question"
    columns = joy_question_column
    empathy_sentences = 'joy_empathy_sentences'
  else:
    additional_questions = 'base_sentences'
    base_question = "base_questions"
    columns = additional_questions_columns
    empathy_sentences = "empathy_sentences"


  internal_dict.setdefault(additional_questions,[])
  internal_dict.setdefault(empathy_sentences,[])
  for index, column in enumerate(columns):
    print(columns)
    corpus = emotion_filtered['train'][column]
    base_utterance, empathy_utterance = split_emp_base(corpus, base_utterance=split_utterances[base_question][index])
    internal_dict[additional_questions].append(base_utterance)
    internal_dict[empathy_sentences].extend(empathy_utterance)
  split_utterances[emotion] = internal_dict
  
  internal_dict_initial = dict()
  internal_dict_initial.setdefault('initial_base_sentences',[])
  internal_dict_initial.setdefault('initial_empathy_sentences',[])
  for index, column in enumerate(initial_questions_columns):
    corpus = emotion_filtered['train'][column]
    base_utterance, empathy_utterance = split_emp_base(corpus, base_utterance=split_utterances["initial_base_questions"][index])
    internal_dict_initial['initial_base_sentences'].append(base_utterance)
    internal_dict_initial['initial_empathy_sentences'].append(empathy_utterance)
  
  internal_dict.update(internal_dict_initial)
  split_utterances[emotion] = internal_dict


print(split_utterances)

['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?']
['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversati

['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?']
['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversati

['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?']
['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversati

['Would you like to attempt protocols for joy?']
{'base_questions': ['вы испытываете к кому-либо злость, зависть или агрессию?', 'Вы чуствуете ответственность за кого-либо?', 'Вам кажется, что вы хотите кого то контролировать?', 'Вы испытываете ощущение жертвы и вините кого то в своём состоянии?', 'Считаете ли вы себя причиной проблемы?', 'Как часто Вы настаиваете на своей позиции не принимая во внимание позицию собеседников?', 'Есть ли у вас сложности с друзьями, возможно в личной жизни?'], 'initial_base_questions': ['Вы полагаете, что ваше состояние связано с каким то конкретным событием?', 'Эта проблема возникла у вас недавно?', 'Методика 6 вызвала у Вас какие-то плохие чувства?', 'Методика 11 вызвала у Вас какие-то плохие чувства?', 'я задам вам несколько вопросов?'], 'joy_question': ['Я бы хотел предложить Вам одно упражнение'], 'joy': {'joy_base_sentences': [[' У меня есть новое интересное задание для вас, чтобы поддержать положительную динамику', ' Чтобы закрепить, могу посовето

In [110]:
split_utterances['joy']['initial_empathy_sentences']

[[], [], [], [], []]

## Cluster split empathy and base sentences. Generate sentences

In [93]:
def cluster(num_clusters, corpus):
  embedder = SentenceTransformer('all-MiniLM-L6-v2')
  corpus_embeddings = embedder.encode(corpus)

  # Perform kmean clustering
  clustering_model = KMeans(n_clusters=num_clusters)
  clustering_model.fit(corpus_embeddings)
  cluster_assignment = clustering_model.labels_
  clustered_sentences = [[] for i in range(num_clusters)]
  for sentence_id, cluster_id in enumerate(cluster_assignment):
      clustered_sentences[cluster_id].append(corpus[sentence_id])
  # for i, cluster in enumerate(clustered_sentences):
  #     print("Cluster ", i+1)
  #     print(cluster)
  #     print("")
  return clustered_sentences

## Generate utterances

In [94]:
def permute_sentences(corpus, num_clusters):
  base_utterances, empathy_utterances = split_emp_base(corpus, base_utterance="Ваше состояние связано с каким-либо конкретным событием?")
  base_clusters = cluster(num_clusters, base_utterances)
  empathy_clusters = cluster(num_clusters, empathy_utterances)
  i = 0
  permutations = []
  for cluster_group in base_clusters:
    for base_sentence in cluster_group:
      for _ in range(2):
        empathy_sentence = random.choice(empathy_clusters[(i - i//10*10)])
        synthetic_sentence = ".".join((empathy_sentence, base_sentence)) + "?"
        i += 1
        permutations.append(synthetic_sentence)
  # downsized_base = select_one_from_cluster(base_clusters)
  # downsized_empathy = select_one_from_cluster(empathy_clusters)
  # permutations = permute(downsized_empathy, downsized_base)
  return permutations

In [98]:
df_dict = dict()
df_dict

{}

In [111]:
for emotion in emotions:
  df_dict[emotion] = dict()
  d = split_utterances[emotion]
  # generate sentences for additional questions
  if emotion != "joy":
    for index, base in enumerate(additional_questions_columns):
      # print(index, base)
      base_utterances = d['base_sentences'][index]
      empathy_utterances = d['empathy_sentences']
      base_clusters = cluster(num_clusters=10, corpus=base_utterances)
      empathy_clusters = cluster(num_clusters=10, corpus=empathy_utterances)
      i = 0
      permutations = []
      for cluster_group in base_clusters:
        for base_sentence in cluster_group:
          for _ in range(2):
            empathy_sentence = random.choice(empathy_clusters[(i - i//10*10)])
            synthetic_sentence = ".".join((empathy_sentence, base_sentence)) + "?"
            i += 1
            permutations.append(synthetic_sentence)
      df_dict[emotion][base] = permutations
  # generate sentences for initial questions
    for index, base in enumerate(initial_questions_columns):
      # print(index, base)
      base_utterances = d['initial_base_sentences'][index]
      empathy_utterances = d['initial_empathy_sentences'][index]
      base_clusters = cluster(num_clusters=10, corpus=base_utterances)
      empathy_clusters = cluster(num_clusters=10, corpus=empathy_utterances)
      i = 0
      permutations = []
      for cluster_group in base_clusters:
        for base_sentence in cluster_group:
          for _ in range(2):
            while len(empathy_clusters[(i - i//10*10)])==0:
              i += 1
            empathy_sentence = random.choice(empathy_clusters[(i - i//10*10)])
            synthetic_sentence = ".".join((empathy_sentence, base_sentence)) + "?"
            i += 1
            permutations.append(synthetic_sentence)
      df_dict[emotion][base] = permutations
  else:
    for index, base in enumerate(joy_question_column):
      # print(index, base)
      base_utterances = d['joy_base_sentences'][index]
      empathy_utterances = d['joy_empathy_sentences']
      base_clusters = cluster(num_clusters=10, corpus=base_utterances)
      print(empathy_utterances)
      empathy_clusters = cluster(num_clusters=10, corpus=empathy_utterances)
      i = 0
      permutations = []
      for cluster_group in base_clusters:
        for base_sentence in cluster_group:
          for _ in range(2):
            while len(empathy_clusters[(i - i//10*10)])==0:
              i += 1
            empathy_sentence = random.choice(empathy_clusters[(i - i//10*10)])
            synthetic_sentence = ".".join((empathy_sentence, base_sentence)) + "?"
            i += 1
            permutations.append(synthetic_sentence)
      df_dict[emotion][base] = permutations
      # print(df_dict[emotion][base])

  

      # print(df_dict[emotion][base])

  
  # print(df_dict[emotion])
  



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  import sys


['Рад видеть, что вы сегодня с хорошим настроением', 'Рад за вас, вижу, что светитесь', 'Очень рад, что у Вас всё хорошо', 'Я очень рад, что у вас всё хорошо', 'Это замечательно, что у вас такое замечательное настроение', 'Я рад Вас видеть в таком радостном настроении', 'Я рада, что у Вас такое хорошее настроение', 'Я очень рада, что у вас все прекрасно', 'Вы меня очень радуете, мне очень приятно смотреть на то как ваше настроение меняется в лучшую сторону', 'Мне радостно понимать, что у вас хорошее настроение и поднятый дух', 'Очень приятно что вы пришли ко мне с таким настроением', 'Я безумно рад что мои советы вам помогли стать счастливым и брести свою вторую половинку', 'Я рад это слышать', 'Я очень рад что вы  себя хорошо чувствуете', 'Я рад что у Вас всё прекрасно', 'Мне очень приятно слышать ,что вы в хорошем настроение ) Я бы рекомендовал массаж для расслабления ', 'Замечательно что у вас все хорошо', 'Я очень рад, что вы в прекрасном настроении', 'Я вас поздравляю и разделяю в

In [112]:
# equalise a length of utterances per emotion
def make_equal_length(dictionary):
  min_rows = min([len(dictionary[base]) for base in dictionary])
  for key in dictionary:
    dictionary[key] = dictionary.get(key)[:min_rows]
  return dictionary

In [113]:
# build a dataset with synthetic sentences
synthetic_df_columns = additional_questions_columns+initial_questions_columns+['emotion']
synthetic_df = pd.DataFrame(columns=synthetic_df_columns)

for emotion in df_dict:
  # print(df_dict[emotion])
  emotion_dict = make_equal_length(df_dict[emotion])
  df = pd.DataFrame.from_dict(emotion_dict)
  df['emotion'] = [emotion for i in range(len(df))]
  synthetic_df = pd.concat([synthetic_df, df])

In [114]:
# check
synthetic_df.head()

,Have you strongly felt or expressed any of the following emotions towards someone?,Do you believe you should be the savior of someone else?,Do you feel that you are trying to control someone?,"Do you see yourself as the victim, blaming someone else for how negative you feel?",Are you always blaming and accusing yourself for when something goes wrong?,Is it possible that in previous conversations you may not have always considerdother viewpoints presented?,Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?,Was this emotion triggered by a specific event?,Was the event recent?,Did you find protocol 6 distressing?,Did you find protocol 11 distressing?,Is it ok to ask additional questions?,emotion,Would you like to attempt protocols for joy?
0,"Спасибо, что вы здесь с нами.как вы считаете и...","Я понимаю, это не самый легкий вопрос.Бывает л...","спасибо за откровенность.Скажите пожалуйста, в...",Благодарю за открытость.Возможно вы оказались ...,Я понимаю вопрос не из лёгких.Как часто вы руг...,Спасибо за доверие.Вы часто настаиваете на том...,"Понимаю, что на этот вопрос нелегко ответить.Б...","Ценю Вашу открытость.Можете ли вы вспомнить, в...","Спасибо, что обратились именно ко мне. Ваше чу...","Спасибо,что вы поделились со мной своими ощуще...","Сочувствую вам, и вашему состоянию.Я вам буду ...",Мне нужно лучше узнать ситуацию.Вынужден задат...,fear,NaN
1,"Я рад, что вы мне доверяете.как вы считаете ис...","Спасибо за доверие.Бывает ли у Вас чувство, чт...","Я понимаю, это не самый легкий вопрос.Скажите ...",Спасибо за доверие.Возможно вы оказались в пол...,"Спасибо за доверие.Как часто вы ругаете себя, ...",Спасибо за искренность.Вы часто настаиваете на...,Благодарю вас.Благодарю вас за оказанное довер...,"Спасибо, что поделились своими чувствами.Может...",Я очень рада что ты решил поделиться со мной с...,"Спасибо, продолжим наш диалог. Возникли ли у В...","Благодарю за то, что поделились своими мыслями...","Я буду задавать Вам вопросы, а Вы постарайтесь...",fear,NaN
2,По всей видимости это нелегко. Испытывали ли в...,"Не кажется ли вам, что вы недостаточно контрол...","Наш с Вами разговор будет более плодотворным, ...","Спасибо, что были открыты со мной. И если у Ва...","Я понимаю, что это трудно.Когда у вас что-то н...","Я думаю это непростой вопрос, но все же попыта...","Спасибо, что доверились мне. Я бы хотел узнат...","Задумайтесь, пожалуста, над истоком Вашего чув...","Я очень вас понимаю и сочувствую, и постараюсь...",Я очень вас понимаю и хочу помочь вам. Были ли...,"Жаль, что не всё так хорошо, спасибо за откров...",Приятно что вы доверились мне.Хотелось бы лучш...,fear,NaN
3,Благодарю за искренность. Испытывали ли вы к к...,"Спасибо, что верите мне. А возникает ли у вас ...","Я очень рад, что вы мне доверились.Нет ли у ва...",Проанализируйте пожалуйста и откровенно ответь...,Я задам Вам непростой вопрос.Когда у вас что-т...,"Благодарю за искренность. Случается ли у Вас, ...","Не бывает ли у вас ""комплекса героя"", желания ...","Спасибо ща ответ.Вы полагаете , что чувство тр...",Я понимаю Вас большего кто бы то не был. А это...,"Благодарен за Ваше доверие и откровенность, оч...","Мне невероятно жалко, что Вы испытываете подоб...",Давайте постараемся более детально разобраться...,fear,NaN
4,Огромное спасибо за откровение. Вы ранее испыт...,"Спасибо за откровенность. Бывает ли такое, что...","Спасибо за то, что открылись мне.Бывало ли у в...",Не просто об этом говорить.У вас возникало чув...,Благодарю за доверие.Вам приходится обвинять с...,Благодарю за доверие. Как часто Вы не слышите ...,"Спасибо, что были открыты со мной. Хотел поинт...","Спасибо, что уточнили Ваше ощущение.Чувство тр...","Благодарен, что доверили мне ваши переживания....",Спасибо ещё раз за Вашу откровенность со мной....,"Благодарен за Ваше доверие и откровенность, оч...","Вы не будете возражать, если я Вам задам неско...",fear,NaN


# Convert pandas into a Huggingface dataset

In [117]:
dataset = Dataset.from_pandas(synthetic_df)

In [118]:
dataset = dataset.remove_columns('__index_level_0__')

# Save a dataset

In [119]:
dataset.save_to_disk("/content/drive/MyDrive/chatbot/SyntheticResponseGenerationDataset")

In [120]:
# check that it loaded correctly
from datasets.load import load_from_disk
ds = load_from_disk("/content/drive/MyDrive/chatbot/SyntheticResponseGenerationDataset")

In [121]:
ds

Dataset({
    features: ['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'emotion', 'Would you like to attempt protocols for joy?'],
    num_rows: 786
})